In [1]:
# install packages
# !pip install -q tensorboard==2.13.0 \
#     tensorflow==2.13.0 \
#     pyarrow==12.0.1 \
#     polars==0.18.6 \
#     tqdm==4.65.0 \
#     scipy==1.10.1 \
#     scikit-learn==1.3.0 \
#     numpy==1.24.3 \
#     Pillow==10.0.0

In [2]:
import os
import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm

from typing import List

import numpy as np
import scipy.sparse as sp
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

from PIL import Image
import requests
import matplotlib.pyplot as plt

In [3]:
movies_df = pl.read_parquet('imdb_movies.parquet')
movies_df

title_id,poster_url,type,name,original_name,description,genre,date,rating_count,rating_value,keywords,featured_review,stars,directors,creators
str,str,str,str,str,str,list[str],str,f64,f64,list[str],str,list[str],list[str],list[str]
"""tt0111161""","""https://m.medi…","""Movie""","""Побег из Шоуше…","""The Shawshank …","""Over the cours…","[""Drama""]","""1994-10-14""",2.755277e6,9.3,"[""prison"", ""based on the works of stephen king"", … ""voice over narration""]","""It is no wonde…","[""Tim Robbins"", ""Morgan Freeman"", ""Bob Gunton""]","[""Frank Darabont""]","[""Stephen King"", ""Frank Darabont""]"
"""tt0068646""","""https://m.medi…","""Movie""","""Крестный отец""","""The Godfather""","""Don Vito Corle…","[""Crime"", ""Drama""]","""1972-03-24""",1.917037e6,9.2,"[""mafia"", ""patriarch"", … ""rise to power""]","""I love this mo…","[""Marlon Brando"", ""Al Pacino"", ""James Caan""]","[""Francis Ford Coppola""]","[""Mario Puzo"", ""Francis Ford Coppola""]"
"""tt0468569""","""https://m.medi…","""Movie""","""Тёмный рыцарь""","""The Dark Knigh…","""When the menac…","[""Action"", ""Crime"", ""Drama""]","""2008-08-14""",2.728169e6,9.0,"[""dc comics"", ""psychopath"", … ""clown""]","""If someone els…","[""Christian Bale"", ""Heath Ledger"", ""Aaron Eckhart""]","[""Christopher Nolan""]","[""Jonathan Nolan"", ""Christopher Nolan"", ""David S. Goyer""]"
"""tt0050083""","""https://m.medi…","""Movie""","""12 разгневанны…","""12 Angry Men""","""The jury in a …","[""Crime"", ""Drama""]","""1961-11-13""",816234.0,9.0,"[""jury"", ""dialogue driven"", … ""jury room""]","""&apos;12 Angry…","[""Henry Fonda"", ""Lee J. Cobb"", ""Martin Balsam""]","[""Sidney Lumet""]","[""Reginald Rose""]"
"""tt0071562""","""https://m.medi…","""Movie""","""Крестный отец …","""The Godfather …","""The early life…","[""Crime"", ""Drama""]","""1974-12-18""",1.305122e6,9.0,"[""revenge"", ""1950s"", … ""lake tahoe nevada""]","""The original G…","[""Al Pacino"", ""Robert De Niro"", ""Robert Duvall""]","[""Francis Ford Coppola""]","[""Francis Ford Coppola"", ""Mario Puzo""]"
"""tt0108052""","""https://m.medi…","""Movie""","""Список Шиндлер…","""Schindler&apos…","""In German-occu…","[""Biography"", ""Drama"", ""History""]","""1994-05-21""",1.388917e6,9.0,"[""holocaust"", ""nazi"", … ""german""]","""This Movie was…","[""Liam Neeson"", ""Ralph Fiennes"", ""Ben Kingsley""]","[""Steven Spielberg""]","[""Thomas Keneally"", ""Steven Zaillian""]"
"""tt0167260""","""https://m.medi…","""Movie""","""Властелин коле…","""The Lord of th…","""Gandalf and Ar…","[""Action"", ""Adventure"", ""Drama""]","""2004-01-22""",1.892435e6,9.0,"[""journey"", ""orc"", … ""ring""]","""Wonderful on e…","[""Elijah Wood"", ""Viggo Mortensen"", ""Ian McKellen""]","[""Peter Jackson""]","[""J.R.R. Tolkien"", ""Fran Walsh"", ""Philippa Boyens""]"
"""tt0110912""","""https://m.medi…","""Movie""","""Криминальное ч…","""Pulp Fiction""","""The lives of t…","[""Crime"", ""Drama""]","""1995-09-29""",2.115956e6,8.9,"[""nonlinear timeline"", ""overdose"", … ""drug dealing""]","""I can only spe…","[""John Travolta"", ""Uma Thurman"", ""Samuel L. Jackson""]","[""Quentin Tarantino""]","[""Quentin Tarantino"", ""Roger Avary""]"
"""tt0120737""","""https://m.medi…","""Movie""","""Властелин коле…","""The Lord of th…","""A meek Hobbit …","[""Action"", ""Adventure"", ""Drama""]","""2002-03-01""",1.920951e6,8.8,"[""ring"", ""quest"", … ""elf""]","""Here is one fi…","[""Elijah Wood"", ""Ian McKellen"", ""Orlando Bloom""]","[""Peter Jackson""]","[""J.R.R. Tolkien"", ""Fran Walsh"", ""Philippa Boyens""]"


In [5]:
movies_df = movies_df.with_columns(
    # далее будем работать с признаками как со списком значений
    pl.col("type").apply(lambda x: [x]).alias("type"),
    # создадим колонку date_years_4, которая содержит диапазон лет, по которым мы будем искать похожие тайтлы
    pl.col("date")
    .apply(lambda x: [str(v) for v in range(int(x[:4]) - 4, int(x[:4]) + 4 + 1)])
    .alias("date_years_4"),
)

## Data Description

For solving the task, the following data is available from the file `imdb_movies.parquet`

| Field           | Type                 | Description                                    |
|---              |---                   |---                                             |
| title_id        | str                  | IMDb identifier                                |
| poster_url      | str                  | Link to the poster                             |
| type            | str                  | Title type: Movie or TVSeries                  |
| name            | str                  | Title name in Russian                          |
| original_name   | str                  | Original title name                            |
| description     | str                  | IMDb description of the title                  |
| genre           | list[str]            | List of genres                                 |
| date            | str                  | Release date of the title                      |
| rating_count    | int                  | Number of reviews for the title                |
| rating_value    | float                | Average rating of the title                    |
| keywords        | list[str]            | Keywords for the title (generated by IMDb users) |
| featured_review | str                  | Featured review for the title                  |
| stars           | list[str]            | List of key actors                             |
| directors       | list[str]            | List of directors                              |
| creators        | list[str]            | List of creators                               |


## Evaluation

The quality metric used is hitrate@10, which can be interpreted as **the probability that at least one of the top 10 recommended items is relevant**. To achieve the maximum score, it is sufficient to reach a hitrate@10 of 0.3.

As `y_relevant`, the titles that appear together in user ratings are used.

In [6]:
TOP_K = 10
SUBMISSION_PATH = "submission.parquet"
RELEVANT_TITLES_PATH = "relevant_titles_subsample.parquet"


def hitrate(y_relevant: List[str], y_preds: List[str], k: int = TOP_K) -> int:
    return int(len(set(y_relevant).intersection(y_preds[:k])) > 0)


# код для подсчета метрики качества
def print_score():
    hitrate_list = []
    # загрузим предсказания в словарь title -> [title_id]
    user_preds = {
        title_id: recs for title_id, recs in pl.read_parquet(SUBMISSION_PATH).rows()
    }
    for title_id, relevant_items in pl.read_parquet(RELEVANT_TITLES_PATH).rows():
        # обрежем рекомендации до top-K значений
        recommended_titles = user_preds.get(title_id, [])[:TOP_K]
        hitrate_list.append(hitrate(relevant_items, recommended_titles))

    mean_hitrate = float(np.mean(hitrate_list))
    print(f"HITRATE@10 = {mean_hitrate}")

## Building Recommendations Based on Categorical Features

For each feature, we will first construct a sparse matrix $A$, where $A_{i, j} = 1$ if feature $j$ is related to title $i$.

After that, we will build a similarity matrix using the cosine distance metric.

In [20]:
similarities_by_col = {}
feature_columns = [
    "date_years_4",
    "type",
    "keywords",
    "genre",
    "directors",
    "creators",
    "stars",
]

def flatten_list(nested_list):
    """ Flatten a nested list """
    for element in nested_list:
        if isinstance(element, list):
            yield from flatten_list(element)
        else:
            yield element

for col in tqdm(feature_columns):
    ## Flatten the list for all columns, handling None and nested list values
    column_data = movies_df[col].to_list()
    flattened_list = [item for sublist in column_data if sublist is not None for item in flatten_list(sublist)]
    unique_values = pd.Series(flattened_list).unique().tolist()

    # Create the mapping
    mapping = {k: v for v, k in enumerate(unique_values)}


    # построим разреженную матрицу признаков для каждого тайтла
    # Construct the sparse matrix
    rows = []
    cols = []
    values = []
    for row_ind, value in enumerate(column_data):
        if value is None:
            value = []

        # Ensure each item in value is not a list
        processed_value = []
        for item in value:
            if isinstance(item, list):
                processed_value.extend(item)  # Flatten nested lists
            else:
                processed_value.append(item)

        col_inds = [mapping[x] for x in processed_value]
        rows.extend([row_ind] * len(col_inds))
        values.extend([1] * len(col_inds))
        cols.extend(col_inds)

    # Construct similarity matrix using cosine distance
    sparse_data = sp.csr_matrix((values, (rows, cols)))
    sparse_data = normalize(sparse_data, norm="l2", axis=1)
    similarities_by_col[col] = (sparse_data @ sparse_data.T).A

100%|██████████| 7/7 [00:03<00:00,  2.05it/s]


## Adding Text Features

We'll encode them in the simplest way, using tf-idf.

We could have used more advanced methods like w2v or OpenAI embeddings, but even such a simple solution allows for the consideration of identical words in two reviews or descriptions of titles.

In [26]:
# Replace None with empty strings for text processing in Polars
movies_df = movies_df.with_columns([
    pl.col('featured_review').fill_null('').alias('featured_review'),
    pl.col('description').fill_null('').alias('description')
])

# Construct the TF-IDF matrix for featured_review
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 2), norm='l2')
sparse_data = vectorizer.fit_transform(movies_df['featured_review'].to_list())
similarities_by_col['featured_review'] = (sparse_data @ sparse_data.T).A

# Construct the TF-IDF matrix for description
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 2), norm='l2')
sparse_data = vectorizer.fit_transform(movies_df['description'].to_list())
similarities_by_col['description'] = (sparse_data @ sparse_data.T).A


## Building Recommendations as an Ensemble

To do this, we will sum up all the similarities with weights, i.e., $sim(title_i, title_j) = \sum_k sim_k(title_i, title_j)$, where $sim_k$ is the obtained similarity matrix for feature $k$.

In [27]:
N = len(movies_df)

similarities = np.zeros((N, N))
for k, v in tqdm(similarities_by_col.items()):
    weight = 1  # каждый признак имеет одинаковую важность
    # итоговая похожесть -- усреднение похожести по каждому признаку
    similarities += weight * (v - np.eye(N))  # вычитаем 

similarities.shape

100%|██████████| 9/9 [00:11<00:00,  1.23s/it]


(9798, 9798)

In [28]:
def get_recommendations(title_ind: int, k: int = 10):
    # для рекомендации похожих тайтлов к title_ind, отсортируем похожести
    # similarities[title_ind] по убыванию и возьмем первые k индексов
    nearest_inds = np.argsort(similarities[title_ind])[::-1][:k]
    return movies_df[nearest_inds]["title_id"].to_list()


TOP_K = 10

submission = []
for title_ind in tqdm(range(len(movies_df))):
    title_id = movies_df["title_id"][title_ind]
    recommended_titles = get_recommendations(title_ind, TOP_K)
    submission.append((title_id, recommended_titles))

pl.DataFrame(submission, schema=("title_id", "recs")).write_parquet(
    "submission.parquet"
)
print_score()

100%|██████████| 9798/9798 [00:10<00:00, 972.69it/s]


HITRATE@10 = 0.321


🚀🚀🚀

## Incorporating Averaging with Respect to Feature Scores

In [44]:
#  Building Recommendations Based on a Single Feature and Calculating the Score
feature_to_score = {
    'date_years_4': 1,
    'type': 1,
    'keywords': 1,
    'genre': 1,
    'directors': 1,
    'creators': 1,
    'stars': 2,
    'featured_review': 2,
    'description': 2,
}

In [45]:
N = len(movies_df)

similarities = np.zeros((N, N))
for k, v in tqdm(similarities_by_col.items()):
    weight = feature_to_score[k]  # используем скор признака как его вес в итоговом ансамбле
    similarities += weight * (v - np.eye(N))

similarities.shape

100%|██████████| 9/9 [00:10<00:00,  1.19s/it]


(9798, 9798)

In [49]:
def get_recommendations(title_ind: int, k: int = 10):
    nearest_inds = np.argsort(similarities[title_ind])[::-1][:k]
    return movies_df[nearest_inds]['title_id'].to_list()


TOP_K = 10

submission = []
for title_ind in tqdm(range(len(movies_df))):
    title_id = movies_df['title_id'][title_ind]
    recommended_titles = get_recommendations(title_ind, TOP_K)
    submission.append((title_id, recommended_titles))
    
pl.DataFrame(submission, schema=('title_id', 'recs')).write_parquet('submission.parquet')
print_score()

100%|██████████| 9798/9798 [00:10<00:00, 963.82it/s]


HITRATE@10 = 0.338
